# YOUTUBE SENTIMENT ANALYSIS

## IF 4072 - Pemrosesan Bahasa Alami

* Jason Wiguna - 13516024 
* Kevin Basuki - 13516071
* Nicolaus Boby - 13516077
* Eric Jonathan -

In [1]:
from emoji import UNICODE_EMOJI, emoji_lis
import emojis
import pandas as pd
import numpy as np
import math

In [2]:
# Kralj Novak, Petra; Smailović, Jasmina; Sluban, Borut and Mozetič, Igor, 2015, Emoji Sentiment Ranking 1.0, Slovenian language resource repository CLARIN.SI, http://hdl.handle.net/11356/1048.
emo_df = pd.read_csv("Data/Emoji_Sentiment_Data_v1.0.csv")
emo_df.head()

,Emoji,Unicode codepoint,Occurrences,Position,Negative,Neutral,Positive,Unicode name,Unicode block
0,😂,0x1f602,14622,0.805101,3614,4163,6845,FACE WITH TEARS OF JOY,Emoticons
1,❤,0x2764,8050,0.746943,355,1334,6361,HEAVY BLACK HEART,Dingbats
2,♥,0x2665,7144,0.753806,252,1942,4950,BLACK HEART SUIT,Miscellaneous Symbols
3,😍,0x1f60d,6359,0.765292,329,1390,4640,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
4,😭,0x1f62d,5526,0.803352,2412,1218,1896,LOUDLY CRYING FACE,Emoticons


In [4]:
emo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 9 columns):
Emoji                969 non-null object
Unicode codepoint    969 non-null object
Occurrences          969 non-null int64
Position             969 non-null float64
Negative             969 non-null int64
Neutral              969 non-null int64
Positive             969 non-null int64
Unicode name         969 non-null object
Unicode block        969 non-null object
dtypes: float64(1), int64(4), object(4)
memory usage: 68.2+ KB


In [10]:
emo_df['score'] = (emo_df['Positive'] - emo_df['Negative']) / (emo_df['Negative'] + emo_df['Positive'] + emo_df['Neutral'])

In [11]:
def get_emoji_sentiments(df, text):
    emos = []
    raw = emoji_lis(text)
    for item in raw:
        emos.append(item.values())
    total = 0
    for emo in emos:
        total += float(df.loc[df['Emoji'] == '👍']['score'])
    return np.tanh(total)

In [15]:
data = pd.read_csv('Data/labeled.csv', names=['Comments', 'Label'])
data.head()

,Comments,Label
0,Hnya mungkin iman nya masih goyah dan alasan t...,-0.561180
1,"Kok pada ngributin hidup orang lain, mau pinda...",0.012860
2,Isi suratnya adalah ....\r\nRina nose tidak pe...,0.000000
3,"hmmmm sayang nya rina kalau ampe pindah,ibarat...",0.014891
4,👍👍👍👍,0.975000


In [16]:
comments = data['Comments']
for comment in comments:
    print(get_emoji_sentiments(emo_df, comment))

0.0
0.0
0.0
0.0
0.9697723080410184
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.7795524516578227
0.47933022218387067
0.0
0.0
0.0
0.0
0.47933022218387067
0.0
0.0
0.9164421172809663
0.0
0.9962052023468827
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.7795524516578227
0.7795524516578227
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.7795524516578227
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.47933022218387067
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.7795524516578227
0.47933022218387067
0.0
0.47933022218387067
0.9697723080410184
0.0
0.47933022218387067
0.47933022218387067
0.0
0.0
0.0
0.9164421172809663
0.0
0.9164421172809663
0.0
0.47933022218387067
0.0
0.47933022218387067
0.7795524516578227
0.0
0.0
0.0
0.0
0.0
0.0
0.9962052023468827
0.9697723080410184
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.7795524516578227
0.47933022218387067
0.47933022218387067
0.0
0.47933022218387067
0.0
0.0
0.0
0.9164421172809663
0.0
0.9164421172809663
0

In [18]:
from string import punctuation
import nltk
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(punctuation)])
    return s

data['cleaned'] = data['Comments'].apply(remove_punctuation)

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

data['tokenized_sents'] = data.apply(lambda row: nltk.word_tokenize(row['cleaned']), axis=1)

print(data['tokenized_sents'])

0       [Hnya, mungkin, iman, nya, masih, goyah, dan, ...
1       [Kok, pada, ngributin, hidup, orang, lain, mau...
2       [Isi, suratnya, adalah, Rina, nose, tidak, per...
3       [hmmmm, sayang, nya, rina, kalau, ampe, pindah...
4                                                  [👍👍👍👍]
5                                         [Azka, ganteng]
6       [Yes, om, Deddy, you, have, built, the, monste...
7       [dapat, suami, bule, dia, jadi, pindah, agama,...
8           [apa, yang, di, madsud, air, kejujuran, rina]
9                                [Yg, nonton, 2019, like]
10      [Bang, sebenar, nya, apa, isi, surat, rina, bang]
11                      [Salam, dari, palangkaraya, bang]
12                            [Gaya, mu, itu, lohhhwkwkw]
13           [Jd, bingung, apa, isi, surat, rina, nose😱😲]
14      [Klo, ngedidik, anak, jjr, gwe, sama, dgn, Om,...
15                             [Om, kapan, punya, rambut]
16                               [Isi, surat, mulai, 630]
17            

In [19]:
from nltk.tag import CRFTagger
import pycrfsuite
# nltk.download('tag')

ct = nltk.tag.CRFTagger()

In [20]:
ct.set_model_file('all_indo_man_tag_corpus_model.crf.tagger')
hasil = ct.tag_sents(data['tokenized_sents'])
hasil

[[('Hnya', 'RB'),
  ('mungkin', 'MD'),
  ('iman', 'VB'),
  ('nya', 'PRP'),
  ('masih', 'MD'),
  ('goyah', 'VB'),
  ('dan', 'CC'),
  ('alasan', 'NN'),
  ('tertntu', 'NN'),
  ('dgn', 'NN'),
  ('melepas', 'VB'),
  ('hijab', 'NN'),
  ('Doakn', 'NNP'),
  ('semoga', 'NNP'),
  ('Rina', 'NNP'),
  ('kembali', 'VB'),
  ('mendapatkn', 'NN'),
  ('kebaikn', 'NN'),
  ('lagi', 'RB'),
  ('Karena', 'SC'),
  ('dia', 'PRP'),
  ('sedang', 'MD'),
  ('di', 'IN'),
  ('Uji', 'NNP')],
 [('Kok', 'NN'),
  ('pada', 'IN'),
  ('ngributin', 'NN'),
  ('hidup', 'NN'),
  ('orang', 'NN'),
  ('lain', 'JJ'),
  ('mau', 'RB'),
  ('pindah', 'VB'),
  ('agama', 'NN'),
  ('kek', 'NN'),
  ('mau', 'RB'),
  ('buka', 'VB'),
  ('hijab', 'NN'),
  ('kek', 'NN'),
  ('mau', 'RB'),
  ('ngapain', 'JJ'),
  ('saja', 'RB'),
  ('kek', 'NN'),
  ('itu', 'PR'),
  ('kan', 'RP'),
  ('hidup2', 'CD'),
  ('mereka', 'PRP'),
  ('kok', 'VB'),
  ('klian', 'NN'),
  ('yg', 'NN'),
  ('kepoh', 'NN'),
  ('toh', 'NN'),
  ('makan', 'VB'),
  ('pun', 'RP'),
  ('N